# とりあえずまばら性ということは置いておき、評価数のある程度多いユーザにレコメンドする。

In [1]:
import pandas as pd
import sys;sys.path.append('..')
import warnings; warnings.simplefilter("ignore")
df = pd.read_csv('../data/explicit_usa.csv')
df.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,2954,0060973129,8.0,71.0,wichita,kansas,usa,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,1,35704,0374157065,6.0,53.0,kansas city,missouri,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
2,2,110912,0374157065,10.0,36.0,milpitas,california,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
3,3,157969,0374157065,8.0,30.0,denver,colorado,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,4,192665,0374157065,8.0,47.0,vacaville,california,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...


In [2]:
from surprise import SVD, Reader, Dataset
from collections import defaultdict
import numpy as np

In [3]:
#filtered_df = df.groupby('ISBN').filter(lambda x: len(x["ISBN"]) >= 50)

In [4]:
#df = df.groupby('User-ID').filter(lambda x: len(x["User-ID"])>=20)
#df = df.reset_index(drop=True)
df = df.drop("Unnamed: 0", axis=1)
#df.groupby('ISBN').filter(lambda x: len(x["ISBN"]) >= 50)

In [5]:
df.head()

,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2954,0060973129,8.0,71.0,wichita,kansas,usa,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,35704,0374157065,6.0,53.0,kansas city,missouri,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
2,110912,0374157065,10.0,36.0,milpitas,california,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
3,157969,0374157065,8.0,30.0,denver,colorado,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,192665,0374157065,8.0,47.0,vacaville,california,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...


In [6]:
from util.train_test_split import split

train_df, test_df = split(df=df)

In [13]:
print(f"trainデータサイズ：　{len(train_df)}")
print(f"testデータサイズ：　{len(test_df)}")

trainデータサイズ：　108016
testデータサイズ：　8545


In [15]:
train_df.head()

,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,35704,0374157065,6.0,53.0,kansas city,missouri,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
1,110912,0374157065,10.0,36.0,milpitas,california,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
2,197659,0374157065,9.0,49.0,indiana,pennsylvania,usa,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
3,29526,0399135782,9.0,26.0,knoxville,tennessee,usa,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
4,46398,0399135782,9.0,37.0,san antonio,texas,usa,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...


In [16]:
test_df.head()

,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
64988,35704,0375508562,9.0,53.0,kansas city,missouri,usa,The Demon in the Freezer : A True Story,RICHARD PRESTON,2002,Random House,http://images.amazon.com/images/P/0375508562.0...,http://images.amazon.com/images/P/0375508562.0...,http://images.amazon.com/images/P/0375508562.0...
47195,35704,0671755056,7.0,53.0,kansas city,missouri,usa,SHAMPOO PLANET,Douglas Coupland,1992,Atria,http://images.amazon.com/images/P/0671755056.0...,http://images.amazon.com/images/P/0671755056.0...,http://images.amazon.com/images/P/0671755056.0...
66933,35704,0399137378,6.0,53.0,kansas city,missouri,usa,Timequake,Kurt Vonnegut,1997,G. P. Putnam's Sons,http://images.amazon.com/images/P/0399137378.0...,http://images.amazon.com/images/P/0399137378.0...,http://images.amazon.com/images/P/0399137378.0...
6507,35704,0385720920,8.0,53.0,kansas city,missouri,usa,Choke,CHUCK PALAHNIUK,2002,Anchor,http://images.amazon.com/images/P/0385720920.0...,http://images.amazon.com/images/P/0385720920.0...,http://images.amazon.com/images/P/0385720920.0...
17182,35704,0671874349,9.0,53.0,kansas city,missouri,usa,LIFE AFTER GOD : LIFE AFTER GOD,Douglas Coupland,1995,Washington Square Press,http://images.amazon.com/images/P/0671874349.0...,http://images.amazon.com/images/P/0671874349.0...,http://images.amazon.com/images/P/0671874349.0...


In [18]:
train_df.groupby('ISBN').filter(lambda x: len(x["ISBN"])>=50)

,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
179,2766,0446310786,10.0,42.0,frisco,texas,usa,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
180,4017,0446310786,10.0,48.0,new orleans,louisiana,usa,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
181,8253,0446310786,10.0,26.0,tulsa,oklahoma,usa,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
182,21576,0446310786,8.0,31.0,memphis,tennessee,usa,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
184,24194,0446310786,8.0,20.0,palos hills,illinois,usa,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20126,260897,043935806X,9.0,25.0,chicago,illinois,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
20127,262118,043935806X,10.0,25.0,maryville,tennessee,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
20128,271176,043935806X,10.0,39.0,louisville,kentucky,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
20129,271448,043935806X,10.0,27.0,ona,west virginia,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...


In [7]:
user_item_list = list(df[df["User-ID"]==35704].index)
user_item_list

[1,
 6153,
 8852,
 12235,
 13209,
 18601,
 21512,
 24265,
 26328,
 27911,
 33036,
 34406,
 42859,
 46250,
 52987,
 66031,
 67987,
 72168,
 75118,
 75287,
 75416,
 79237,
 83353,
 95280,
 97679,
 110160,
 111142,
 111820,
 113122,
 113474,
 117358,
 122033,
 122098,
 122099,
 122100,
 122101,
 122102,
 122103,
 122117,
 122120,
 122121,
 122122,
 122123,
 122125,
 122129,
 122130]

In [8]:
rate_len = len(user_item_list)
    
random_index = np.random.randint(0, rate_len+1, 5)
test_index = [user_item_list[i] for i in random_index]
test_index

[12235, 97679, 122099, 24265, 122033]

In [9]:
set([1,2,3])- set([1,2])

{3}

In [10]:
a = set([1,2,3])
b = set([1,2])
a.difference(b)


{3}

In [11]:
df.iloc[test_index]

,User-ID,ISBN,Book-Rating,Age,city,state,country,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
12235,35704,0671726889,5.0,53.0,kansas city,missouri,usa,CUCKOO'S EGG,Clifford Stoll,1990,Pocket,http://images.amazon.com/images/P/0671726889.0...,http://images.amazon.com/images/P/0671726889.0...,http://images.amazon.com/images/P/0671726889.0...
97679,35704,0609600761,9.0,53.0,kansas city,missouri,usa,Questioning the Millennium: A Rationalist's Gu...,Stephen Jay Gould,1997,Harmony,http://images.amazon.com/images/P/0609600761.0...,http://images.amazon.com/images/P/0609600761.0...,http://images.amazon.com/images/P/0609600761.0...
122099,35704,0374146608,8.0,53.0,kansas city,missouri,usa,Edwin Hubble: Mariner of the Nebulae,Gale E. Christianson,1995,Farrar Straus &amp; Giroux,http://images.amazon.com/images/P/0374146608.0...,http://images.amazon.com/images/P/0374146608.0...,http://images.amazon.com/images/P/0374146608.0...
24265,35704,0060392452,10.0,53.0,kansas city,missouri,usa,Stupid White Men ...and Other Sorry Excuses fo...,Michael Moore,2002,Regan Books,http://images.amazon.com/images/P/0060392452.0...,http://images.amazon.com/images/P/0060392452.0...,http://images.amazon.com/images/P/0060392452.0...
122033,35704,0385419945,10.0,53.0,kansas city,missouri,usa,Silicon Snake Oil: Second Thoughts on the Info...,Clifford Stoll,1996,Anchor,http://images.amazon.com/images/P/0385419945.0...,http://images.amazon.com/images/P/0385419945.0...,http://images.amazon.com/images/P/0385419945.0...


In [12]:
reader = Reader(rating_scale=(1,10))
data_train = Dataset.load_from_df(
    filterd_df[["User-ID","ISBN","Book-Rating"]], reader
).build_full

NameError: name 'filterd_df' is not defined